# D3TOP - Projeto em Grupo

Participantes:
* Beatriz Andrade Luiz
* Gabriel Marques Molonhoni

## Classificação de comentários de compras no e-commerce

### 1. Motivação

A motivação para esse trabalho é poder classificar os comentários da área do varejo
para que esses dados possam ser aproveitados pelas empresas varejistas brasileiras na hora da
divulgação e oferecimento de promoções, incluindo, uma análise sazonal. Sendo assim, a oferta de
produtos pode se adequar a vontade do consumidor naquela época do ano, fazendo com que o
consumo e a economia varejista possam crescer. Além disso, pode-se oferecer uma análise em
tempo real dessas pesquisas e servindo também como entrada para algoritmos de recomendação.

### 2. Descrição
Esse trabalho busca estudar e implementar uma solução de análise de dados de
mecanismos de pesquisa online para o setor de varejo, fazendo com que o setor varejista possa
oferecer produtos específicos em determinados meses do ano com o intuito de adequar as vendas
às demandas da procura online.

O intuito da busca desse assunto deve-se ao crescimento da procura de produtos de
diversos setores de forma online, onde muitas vezes o consumidor não consegue adquirir o
produto desejado, ou pela falta de promoções atrativas daquele produto, ou pela falta de
divulgações da empresa varejista daquele produto específico. O objetivo é que com essa análise,
as empresas varejistas possam verificar quais são os produtos mais buscados e desejados em
determinada época do ano, para que possa ser feita uma melhor oferta daqueles produtos.

### 3. Base de dados

A base de dados pode ser acessada no Kaggle, através do link https://www.kaggle.com/datasets/jayeshsalunke101/brazilian-ecommerce-public-dataset?resource=download.

Há 8 conjuntos de dados que pode ser agrupados e contém dados de:

* Consumidores
* Vendedores
* Produtos
* Pedidos
* Artigos dos pedidos
* Avaliação de pedidos
* Pagamentos
* Geolocalização

Este trabalho terá foco na avaliação de pedidos e classificação (em positivo e negativo).

### Desenvolvimento

In [18]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np

##### Leitura dos datasets

In [19]:
files = {'customers'    : './dataset/olist_customers_dataset.csv',
         'geolocation'  : './dataset/olist_geolocation_dataset.csv',
         'items'        : './dataset/olist_order_items_dataset.csv',
         'payment'      : './dataset/olist_order_payments_dataset.csv',
         'orders'       : './dataset/olist_orders_dataset.csv',
         'products'     : './dataset/olist_products_dataset.csv',
         'sellers'      : './dataset/olist_sellers_dataset.csv',
         'review'       : './dataset/olist_order_reviews_dataset.csv',
         }

dataframes = {}
for key, value in files.items():
    dataframes[key] = pd.read_csv(value)

##### Cruzamento dos dados (agrupando em único dataframe)

In [20]:
# Cruzamento dos dataframes
df_raw = dataframes['customers'].merge(dataframes['geolocation'], how='inner', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').drop_duplicates('customer_id', keep='first')
df_raw = df_raw.merge(dataframes['orders'], how='inner', on='customer_id')
df_raw = df_raw.merge(dataframes['items'], how='inner', on='order_id')
df_raw = df_raw.merge(dataframes['products'], how='inner', on='product_id')
df_raw= df_raw.merge(dataframes['review'], how='left', on='order_id')

##### Selecionando as colunas de interesse

In [21]:
# Selecionando as colunas de interesse
df_raw = df_raw[['customer_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state',
       'geolocation_lat', 'geolocation_lng','order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp']]


In [22]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113015 entries, 0 to 113014
Data columns (total 27 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    113015 non-null  object 
 1   customer_zip_code_prefix       113015 non-null  int64  
 2   customer_city                  113015 non-null  object 
 3   customer_state                 113015 non-null  object 
 4   geolocation_lat                113015 non-null  float64
 5   geolocation_lng                113015 non-null  float64
 6   order_id                       113015 non-null  object 
 7   order_status                   113015 non-null  object 
 8   order_purchase_timestamp       113015 non-null  object 
 9   order_approved_at              113000 non-null  object 
 10  order_delivered_carrier_date   111819 non-null  object 
 11  order_delivered_customer_date  110554 non-null  object 
 12  order_estimated_delivery_date 

Como podemos ver, as colunas referentes as datas estão como objetos. Vamos converte-las para datetime

In [23]:
# Convertendo para datetime
datas = ['order_purchase_timestamp'
        ,'order_purchase_timestamp'
        ,'order_delivered_carrier_date'
        ,'order_delivered_customer_date'
        ,'order_estimated_delivery_date'
        ,'shipping_limit_date'
        ,'review_creation_date'
        ,'review_answer_timestamp' 
        ]

for data in datas:
    df_raw[data] = pd.to_datetime(df_raw[data])

 Vamos criar uma coluna para o tempo total para entrega (compra -> entrega)

In [24]:
# Criando coluna de tempo de entrega e Hora da compra
df_raw['delivery_time'] = (df_raw['order_delivered_customer_date'].dt.date - df_raw['order_purchase_timestamp'].dt.date).dt.days

In [25]:
# Seleção do período de interesse
df_raw = df_raw[(df_raw['order_purchase_timestamp'].dt.year > 2016)]

In [26]:
df_clean = df_raw.copy()

In [27]:
df_clean = df_clean.reset_index(drop=True)
df_clean.head(10)

,customer_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng,order_id,order_status,order_purchase_timestamp,order_approved_at,...,freight_value,product_category_name,product_photos_qty,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,delivery_time
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,franca,SP,-20.51,-47.40,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,...,21.88,moveis_escritorio,1.00,88b8b52d46df026a9d1ad2136a59b30b,4,NaN,NaN,2017-05-26,2017-05-30 22:34:40,9.00
1,97ee81fede5757f1ab031e83d1a59f56,47800,barreiras,BA,-12.15,-45.00,a4866dfc455353f48014a2b9a86608a2,delivered,2018-01-11 16:47:03,2018-01-11 16:55:38,...,43.26,moveis_escritorio,1.00,e7b977345ddac9a94389b44959e9d929,4,NaN,NaN,2018-02-15,2018-03-15 18:15:00,34.00
2,97ee81fede5757f1ab031e83d1a59f56,47800,barreiras,BA,-12.15,-45.00,a4866dfc455353f48014a2b9a86608a2,delivered,2018-01-11 16:47:03,2018-01-11 16:55:38,...,43.26,moveis_escritorio,1.00,e7b977345ddac9a94389b44959e9d929,4,NaN,NaN,2018-02-15,2018-03-15 18:15:00,34.00
3,44e25679aa4d5d2ef5103f6b0174a586,6414,barueri,SP,-23.50,-46.87,03afb3c10af52ddfa3c33b9881bf43f8,delivered,2017-12-07 16:41:50,2017-12-07 16:51:57,...,18.51,moveis_escritorio,1.00,e934b832e3bd9951f2a163ea80cfe20d,4,NaN,NaN,2017-12-19,2017-12-21 21:48:38,11.00
4,ac318397fb90e64c9a724a3fc61ef587,37137,alfenas,MG,-21.41,-45.94,3c75044db750a89606612ed286713eb8,delivered,2018-03-13 20:11:20,2018-03-13 20:49:24,...,57.08,moveis_escritorio,1.00,b89b4460c0b589c0614d9b0126ae755a,3,NaN,Produto bom em conforme com o preço,2018-04-03,2018-04-03 23:42:32,21.00
5,cf4dc8d4be7642c7c05a73188a3d8c1c,23585,rio de janeiro,RJ,-22.93,-43.64,edc9338008f2e9017f842a8bcad3af94,delivered,2017-04-28 16:55:22,2017-04-28 17:10:12,...,15.04,moveis_escritorio,1.00,51269da957ddb113f61d7ca81418e1f5,5,NaN,NaN,2017-05-17,2017-05-18 23:01:24,18.00
6,ab156e9202420e577b405360a7b29cbc,28950,armacao dos buzios,RJ,-22.77,-41.90,54d6f9d8f56229d3da815add65ec2408,delivered,2018-03-22 10:58:13,2018-03-22 11:15:35,...,57.08,moveis_escritorio,1.00,ba99c73cdfb4cac0b65aecc0d94787bb,1,Negativa,Não recebi o produto e nem satisfação,2018-05-02,2018-05-09 18:56:15,60.00
7,ab156e9202420e577b405360a7b29cbc,28950,armacao dos buzios,RJ,-22.77,-41.90,54d6f9d8f56229d3da815add65ec2408,delivered,2018-03-22 10:58:13,2018-03-22 11:15:35,...,57.08,moveis_escritorio,1.00,ba99c73cdfb4cac0b65aecc0d94787bb,1,Negativa,Não recebi o produto e nem satisfação,2018-05-02,2018-05-09 18:56:15,60.00
8,11eee0e62a0b905cf878ff13c8aec4a0,40230,salvador,BA,-12.99,-38.51,ff7400d904161b62b6e830b3988f5cbd,delivered,2018-03-27 15:37:03,2018-03-27 15:51:53,...,37.97,moveis_escritorio,1.00,e9ccfd7b04e62cf0a6ac4a66b9783886,2,NaN,NaN,2018-04-26,2018-04-27 11:10:05,29.00
9,645d4fd3f4546d7c9592576665711186,17521,marilia,SP,-22.24,-49.92,6b5c76c2c9615b42e0c2b7efb11b60d7,delivered,2018-04-03 21:24:29,2018-04-03 21:35:21,...,23.54,moveis_escritorio,1.00,fbb7c08e8ca00f3412c095449c59d83b,3,NaN,"o produto chegou certinho, porém não veio os p...",2018-04-25,2018-04-25 18:17:53,21.00
